In [1]:
import pandas as pd

In [285]:
import re

In [132]:
from datetime import datetime

In [173]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

# Exploring

In [2]:
historical = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/WNBA-stats/master/wnba-player-stats.csv')

In [6]:
historical.head()

,player_ID,Player,year_ID,Age,Tm,tm_gms,Tm_Net_Rtg,Pos,G,MP,MP_pct,PER,TS_pct,ThrPAr,FTr,ORB_pct,TRB_pct,AST_pct,STL_pct,BLK_pct,TOV_pct,USG_pct,OWS,DWS,WS,WS40,Composite_Rating,Wins_Generated
0,montgre01w,Renee Montgomery,2019,32,ATL,34,-9.8,G,34,949,69.5%,11.1,0.520,0.727,0.176,1.0,4.1,18.0,1.7,0.5,16.8,17.5,0.4,0.5,0.9,0.039,-2.4,1.22
1,williel01w,Elizabeth Williams,2019,26,ATL,34,-9.8,C-F,32,909,66.6%,16.7,0.521,0.000,0.477,11.4,12.1,7.8,1.4,4.7,12.9,15.9,1.6,1.0,2.7,0.117,0.6,2.51
2,sykesbr01w,Brittney Sykes,2019,25,ATL,34,-9.8,G,34,880,64.5%,11.3,0.445,0.308,0.259,3.0,9.1,19.6,1.2,1.5,14.8,23.1,-0.8,0.8,0.0,-0.001,-3.4,0.70
3,hayesti01w,Tiffany Hayes,2019,29,ATL,34,-9.8,G,29,817,59.9%,15.1,0.497,0.313,0.340,2.7,5.7,21.9,1.8,0.8,14.0,27.1,0.5,0.6,1.0,0.050,-1.5,1.45
4,brelaje01w,Jessica Breland,2019,31,ATL,34,-9.8,F,33,767,56.2%,15.2,0.425,0.138,0.120,5.7,16.6,14.2,2.8,3.6,11.6,18.9,-0.4,1.9,1.5,0.076,-0.8,1.62


In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
url = "https://www.basketball-reference.com/wnba/years/2020_advanced.html"
html = requests.get(url)
page = html.content
soup = BeautifulSoup(page, "lxml")

IndexError: list index out of range

In [9]:
tables = soup.find_all('table')

In [16]:
stats2020 = pd.read_html(str(tables[0]))[0]

In [19]:
stats2020[stats2020.Player == 'Alisia Jenkins']

,Player,Tm,Pos,G,MP,PER,TS%,3PAr,FTr,ORB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 16,OWS,DWS,WS,WS/40
146,Alisia Jenkins,TOT,NaN,5,28,-3.5,.376,.375,.750,0.0,8.3,0.0,0.0,0.0,22.0,21.7,NaN,-0.1,0.0,-0.1,-0.193
155,Alisia Jenkins,IND,NaN,1,2,6.7,NaN,NaN,NaN,0.0,30.1,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,.036
163,Alisia Jenkins,CHI,NaN,2,3,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,0.0,-0.006
169,Alisia Jenkins,PHO,NaN,2,23,-4.8,.376,.375,.750,0.0,7.5,0.0,0.0,0.0,22.0,26.4,NaN,-0.1,0.0,-0.1,-0.237


In [22]:
table = BeautifulSoup(str(tables[0]))

In [48]:
url_object = table.findAll('a')[0]

In [50]:
url_object.getText()

'Natalie Achonwa'

In [25]:
stats2020.shape

(170, 21)

# Scraping box scores

In [158]:
def get_team_stats(game_page, team_tables, team, opponent, home, playoffs, link, date):
    team_stats = pd.read_html(str(team_tables[1-home]))[0]
    team_stats.dropna(inplace=True)
    team_stats = team_stats.iloc[:-1].copy()
    team_stats['team'] = team
    team_stats['opp'] = opponent
    team_stats['home_away'] = home
    team_stats['starter'] = [1]*5 + [0]*(team_stats.shape[0]-5)
    team_stats['playoffs'] = playoffs
    team_stats['game_id'] = link.split('.')[0].split('/')[-1]
    team_stats['date'] = datetime.strptime(date, '%a, %b %d, %Y')
    
    team_soup = BeautifulSoup(str(game_page.findAll('table')[1-home]))
    team_stats['player_link'] = [link['href'] for link in team_soup.findAll('a')]
    
    return team_stats

In [159]:
def get_box_scores_year(schedule, playoff_start):
    box_scores = []
    for i, row in schedule.iterrows():
        html = requests.get(f"https://www.basketball-reference.com{row['link']}")
        page = html.content
        game_page = BeautifulSoup(page, "lxml")              
        all_tables = game_page.findAll('table')
        if len(all_tables) == 0:
            schedule.loc[i, 'no_box'] = 1
            continue

        try:
            home_stats = get_team_stats(game_page, all_tables, row['Home/Neutral'], row['Visitor/Neutral'], 
                                        1, i < playoff_start, row['link'], row['Date'])
            away_stats = get_team_stats(game_page, all_tables, row['Visitor/Neutral'], row['Home/Neutral'], 
                                        0, i < playoff_start, row['link'], row['Date'])
        except:
            print(i)
            raise Error          

        box_scores.extend([home_stats, away_stats])
    
    return box_scores

In [176]:
# all_box_scores = []
# all_schedules = []
print(datetime.now())
for year in range(2018, 2021):
    season_schedule_url = f'https://www.basketball-reference.com/wnba/years/{year}-schedule.html'
    html = requests.get(season_schedule_url)
    page = html.content
    schedule_page = BeautifulSoup(page, "lxml")
    schedule = pd.read_html(str(schedule_page.findAll('table')[0]))[0]
    schedule_links = BeautifulSoup(str(schedule_page.findAll('table')[0]))
    
    playoff_start = (schedule.iloc[:,-1] == 'Playoffs').idxmax()
    schedule = schedule[schedule.iloc[:, -1] == 'Box Score'].copy()
    schedule['link'] = [link['href'] for link in schedule_links.findAll('a') if link.text == 'Box Score']
    
    year_box_scores = get_box_scores_year(schedule, playoff_start)
    all_schedules.append(schedule)
    all_box_scores.extend(year_box_scores)
    
    print(year)
    print(datetime.now())

2020-12-04 13:20:02.515816
2018
2020-12-04 13:21:48.018907
2019
2020-12-04 13:23:31.970676
2020
2020-12-04 13:24:43.383519


In [182]:
box_scores_df = pd.concat(all_box_scores, ignore_index=True)

In [184]:
all_schedules_df = pd.concat(all_schedules, ignore_index=True)

/Users/eddieowens/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [191]:
box_scores_df.to_csv('wnba_box_scores.csv', index=False)

In [192]:
all_schedules_df.to_csv('wnba_schedule.csv', index=False)

In [ ]:
# to do
# scrape player info - position, birthday, height, weight, college, draft position, draft year
# scrape odds history

# Scraping odds

In [200]:
from selenium import webdriver
from selenium.webdriver.common.by import By

In [489]:
def scrape_year(year):
    if year == 2020:
        odds_url = "https://www.oddsportal.com/basketball/usa/wnba/results/"
    else:
        odds_url = f"https://www.oddsportal.com/basketball/usa/wnba-{year}/results/"
    
    odds_text = []
    pg_odds_text = get_odds_text(odds_url)
    page_num = 1
    while 'No data available' not in pg_odds_text:
        odds_text.append(pg_odds_text)
        page_num += 1
        pg_url = odds_url + f'#/page/{page_num}/'
        pg_odds_text = get_odds_text(pg_url)
    
    return odds_text

In [478]:
def get_odds_text(odds_url):
    driver = webdriver.Chrome('/usr/local/bin/chromedriver')
    driver.get(odds_url)
    table = driver.find_element_by_id('tournamentTable')
    table_text = table.text
    driver.quit()
    
    return table_text

In [514]:
def get_info_one_game(chunks):
    two_teams = chunks[0][6:]
    first_w = re.search('\\bW\\b', two_teams).span()[0]
    second_w = first_w + re.search('\\bW\\b', two_teams[(first_w+1):]).span()[1]
    
    game_dict = {'team1': two_teams[:first_w-1],
                 'team2': two_teams[first_w+4:second_w-1]}
    if second_w < len(two_teams) - 2:
        score = two_teams[second_w+2:]
        score_chunk = 0
    else:
        score = chunks[1].strip()
        score_chunk = 1
    score = re.sub('OT', '', score)
    try:
        team1_score, team2_score = score.split(':')
        team1_odds = chunks[score_chunk+1]
        team2_odds = chunks[score_chunk+2]
    except ValueError:
        team1_score = 0
        team2_score = 0
        team1_odds = 0
        team2_odds = 0
    game_dict['team1_score'] = int(team1_score)
    game_dict['team2_score'] = int(team2_score)
    try:
        game_dict['team1_odds'] = int(team1_odds)
        game_dict['team2_odds'] = int(team2_odds)
    except ValueError:
        game_dict['team1_odds'] = 0
        game_dict['team2_odds'] = 0
    
    return game_dict, score_chunk+4

In [506]:
def parse_chunks(odds_chunks):
    i = 0
    game_dicts = []
    while i < len(odds_chunks)-1:
        possible_date = odds_chunks[i][:11]
        try:
            game_date = datetime.strptime(possible_date, '%d %b %Y')
            i += 1
        except ValueError:
            game_dict, chunks_read = get_info_one_game(odds_chunks[i:(i+5)])
            game_dict['date'] = game_date
            game_dicts.append(game_dict)
            i += chunks_read
    
    return pd.DataFrame(game_dicts)

In [519]:
# odds_info = []
# for year in range(2009, 2021):
year = 2020
odds_texts = scrape_year(year)
odds_info.extend([parse_chunks(odds_text.split('\n')[5:]) for odds_text in odds_texts])
print(year)

2020


In [520]:
odds_df = pd.concat(odds_info, ignore_index=True)

In [523]:
odds_df.to_csv('odds_data.csv')

# Scraping player info

In [28]:
player_url = 'https://www.basketball-reference.com/wnba/players/a/achonna01w.html'
html = requests.get(player_url)
page = html.content
player_page = BeautifulSoup(page, "lxml")

In [31]:
player_page

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202011111" rel="dns-prefetch"/>
<title>Natalie Achonwa WNBA Stats | Basketball-Reference.com</title>
<meta content="Statistics, history, awards and achievements for WNBA player Natalie Achonwa" name="Description"/>
<link href="https://www.basketball-reference.com/wnba/players/a/achonna01w.html" rel="canonical"/>
<!-- include:start ="/inc/klecko_header_bbr.html_f" -->
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[

In [44]:
span = player_page.findAll('span', {'itemprop': 'birthDate'})[0]

In [47]:
span.getText()

'\n    November 22,\xa01992\n    '